Here we are importing the necessery libraries for our notebook

In [ ]:
!pip install textblob
!pip install spacy
!pip install gensim
!pip install wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import os

from wordcloud import WordCloud


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from bs4 import BeautifulSoup
!pip install contractions
import contractions
from textblob import TextBlob
import spacy


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
drop_cols = ['id','keyword','location']
train = train.drop(drop_cols,axis=1)
test = test.drop(drop_cols,axis=1)

In [ ]:
sns.countplot('target', data = train)
plt.title('Real or not Disaster tweets')
plt.show()

In [ ]:
def remove_tweet_username(df):
    return re.sub('@[^\s]+','', df)

def make_lower(df):
    return df.lower()

def cont_exp(df):
    return contractions.fix(df)

def make_string(df):
    return str(df)

def remove_url(df):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , df)

def remove_email(df):
    return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", df)

#Retweets
def remove_rt(df):
    df = str(df)
    return re.sub(r'\brt\b', "", df).strip()
def remove_html(df):
    return BeautifulSoup(df, 'lxml').get_text().strip()



def remove_special_chars(df):
    df = re.sub(r'[^\w]+', " ", df)
    df = ' '.join(df.split())
    
    return df


def resub(df):
    return re.sub("(.)\\1{2,}", "\\1", df)
def get_clean_data(df):
    df = remove_url(df)
    df = remove_email(df)
    df = remove_special_chars(df)
    df = remove_html(df)
    df = make_lower(df)
    df = make_string(df)
    df = cont_exp(df)
    df = remove_rt(df)
    df = resub(df)                             
    df = remove_tweet_username(df)
    
    return df


In [ ]:
train['text'] = train['text'].apply(get_clean_data)
test['text'] = test['text'].apply(get_clean_data)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

train['text'] = train['text'].apply(remove_stopwords)
test['text'] = test['text'].apply(remove_stopwords)


In [ ]:
train.text

In [ ]:
train['text'] = train['text'].str.replace('\d+', '')
test['text'] = test['text'].str.replace('\d+', '')

In [ ]:
train.head()

In [ ]:
def get_word_freqs(df, col):
    text = ' '.join(df[col])
    text = text.split()
    freq = pd.Series(text).value_counts()
    return freq

In [ ]:
real_data = get_word_freqs(train[train['target']==1], 'text')
real_data = ' '.join(real_data.index)

In [ ]:
wordcloud = WordCloud().generate(real_data)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
not_real_data = get_word_freqs(train[train['target']==0], 'text')
not_real_data = ' '.join(not_real_data.index)

In [ ]:
wordcloud = WordCloud().generate(not_real_data)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
text = train['text']
y = train['target']

In [ ]:
count = CountVectorizer()
X = count.fit_transform(text)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=12)

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model = LinearSVC()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.predict(X_test)

In [ ]:
model.score(X_train,y_train)

In [ ]:
model.score(X_test,y_test)